In [1]:
import requests
import pandas as pd

ticker_price = requests.get('https://api.binance.com/api/v3/ticker/price?symbol=BTCUSDT')
print(ticker_price.json())

{'symbol': 'BTCUSDT', 'price': '94608.70000000'}


In [2]:
tickers_prices = requests.get('https://api.binance.com/api/v3/ticker/price')

pd.DataFrame(tickers_prices.json()).head(20)

,symbol,price
0,ETHBTC,0.01896000
1,LTCBTC,0.00092200
2,BNBBTC,0.00635200
3,NEOBTC,0.00006850
4,QTUMETH,0.00131700
5,EOSETH,0.00039100
6,SNTETH,0.00001700
7,BNTETH,0.00020260
8,BCCBTC,0.00000000
9,GASBTC,0.00003660


In [3]:
def get_klines(symbol: str, interval:str="1d", limit:int=7):
    klines = requests.get(f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&limit={limit}')

    klines_df = pd.DataFrame(klines.json())
    
    klines_df.columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
    klines_df['open_time'] = pd.to_datetime(klines_df['open_time'], unit='ms')
    klines_df['close_time'] = pd.to_datetime(klines_df['close_time'], unit='ms')
    klines_df = klines_df[['open_time', 'close_time', 'volume', 'low', 'close']]
    
    return klines_df

klines_df = get_klines('BTCUSDT')
klines_df

,open_time,close_time,volume,low,close
0,2025-04-20,2025-04-20 23:59:59.999,8091.67725000,83949.52000000,85179.24000000
1,2025-04-21,2025-04-21 23:59:59.999,31773.37262000,85144.76000000,87516.23000000
2,2025-04-22,2025-04-22 23:59:59.999,43872.74705000,87076.03000000,93442.99000000
3,2025-04-23,2025-04-23 23:59:59.999,27404.16808000,91935.41000000,93691.08000000
4,2025-04-24,2025-04-24 23:59:59.999,19497.06071000,91660.01000000,93980.47000000
5,2025-04-25,2025-04-25 23:59:59.999,27500.66648000,92855.96000000,94638.68000000
6,2025-04-26,2025-04-26 23:59:59.999,2897.42611000,94527.84000000,94616.67000000


In [6]:
from datetime import datetime, timedelta
datetime.today() - timedelta(days=7)

datetime.datetime(2025, 4, 19, 7, 45, 44, 56991)

In [11]:
DT_FORMAT = '%Y-%m-%d'

def add_days(dt, days, dt_format=DT_FORMAT):
    return datetime.strftime(datetime.strptime(dt, dt_format) + timedelta(days=days), dt_format)


add_days('2025-04-20', -7)

'2025-04-13'

In [51]:
def get_klines(symbol: str, start_date: str, end_date: str, interval: str = "1d") -> pd.DataFrame:
    sdt = int(datetime.strptime(start_date, DT_FORMAT).timestamp() * 1000)
    edt = int(datetime.strptime(end_date, DT_FORMAT).timestamp() * 1000)

    params = {
        "symbol": symbol,
        "interval": interval,
        "startTime": sdt,
        "endTime": edt
    }
    
    klines = requests.get('https://api.binance.com/api/v3/klines', params=params)
    klines_df = pd.DataFrame(klines.json())

    klines_df.columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume',
                         'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
    klines_df['open_time'] = pd.to_datetime(klines_df['open_time'], unit='ms')
    klines_df['close_time'] = pd.to_datetime(klines_df['close_time'], unit='ms')
    for col in ['open', 'high', 'low', 'close', 'volume']:
        klines_df[col] = klines_df[col].apply(lambda x: float(x))

    klines_df['date'] = klines_df['open_time']
    # klines_df = klines_df[['date', 'close_time', 'volume', 'open', 'close']]
    klines_df['diff'] = klines_df['close'] - klines_df['open']
    klines_df['taker_buy_base_asset_volume'] = klines_df['taker_buy_base_asset_volume'].astype('float')
    return klines_df


end_date = '2025-04-25'
start_date = add_days(end_date, -28)
klines_df = get_klines('BTCUSDT', start_date, end_date)
klines_df.tail(7)

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,date,diff
21,2025-04-18,84947.92,85132.08,84303.96,84474.69,6529.96315,2025-04-18 23:59:59.999,552648483.64748710,840009,2911.78856,246467088.26214660,0,2025-04-18,-473.23
22,2025-04-19,84474.70,85677.99,84364.45,85077.01,9666.58153,2025-04-19 23:59:59.999,823222752.28424930,875226,5416.30087,461342923.99908200,0,2025-04-19,602.31
23,2025-04-20,85077.00,85320.76,83949.52,85179.24,8091.67725,2025-04-20 23:59:59.999,685253644.89183620,944130,3843.48314,325482835.73783290,0,2025-04-20,102.24
24,2025-04-21,85179.24,88465.99,85144.76,87516.23,31773.37262,2025-04-21 23:59:59.999,2773299591.46706560,3041395,17216.39877,1502705225.13413500,0,2025-04-21,2336.99
25,2025-04-22,87516.22,93888.00,87076.03,93442.99,43872.74705,2025-04-22 23:59:59.999,3967785064.12817020,3268475,22777.77047,2062639579.20308550,0,2025-04-22,5926.77
26,2025-04-23,93442.99,94696.05,91935.41,93691.08,27404.16808,2025-04-23 23:59:59.999,2560279933.94795160,3016536,13128.05774,1226992558.57654620,0,2025-04-23,248.09
27,2025-04-24,93691.07,94005.00,91660.01,93980.47,19497.06071,2025-04-24 23:59:59.999,1811120137.31467870,2797383,8565.18128,795787487.36089810,0,2025-04-24,289.40


In [52]:
# Процентное изменение цены
klines_df['pct_change'] = klines_df['close'].pct_change() * 100  

# Среднеисторическая волатильность (скользящее стандартное отклонение)
klines_df['volatility_7d'] = klines_df['pct_change'].rolling(7).std()  

# Скользящие средние (MA)
klines_df['MA_7'] = klines_df['close'].rolling(7).mean()  
klines_df['MA_21'] = klines_df['close'].rolling(21).mean()  

# Разница между ценой и MA (отклонение от тренда)
klines_df['price_ma7_diff'] = klines_df['close'] - klines_df['MA_7']  
klines_df.tail(7)

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,date,diff,pct_change,volatility_7d,MA_7,MA_21,price_ma7_diff
21,2025-04-18,84947.92,85132.08,84303.96,84474.69,6529.96315,2025-04-18 23:59:59.999,552648483.64748710,840009,2911.78856,246467088.26214660,0,2025-04-18,-473.23,-0.557071,1.402898,84389.350000,82675.790000,85.340000
22,2025-04-19,84474.70,85677.99,84364.45,85077.01,9666.58153,2025-04-19 23:59:59.999,823222752.28424930,875226,5416.30087,461342923.99908200,0,2025-04-19,602.31,0.713018,1.127200,84360.794286,82791.431429,716.215714
23,2025-04-20,85077.00,85320.76,83949.52,85179.24,8091.67725,2025-04-20 23:59:59.999,685253644.89183620,944130,3843.48314,325482835.73783290,0,2025-04-20,102.24,0.120162,0.823125,84563.542857,82924.252857,615.697143
24,2025-04-21,85179.24,88465.99,85144.76,87516.23,31773.37262,2025-04-21 23:59:59.999,2773299591.46706560,3041395,17216.39877,1502705225.13413500,0,2025-04-21,2336.99,2.743615,1.246152,84981.350000,83160.739524,2534.880000
25,2025-04-22,87516.22,93888.00,87076.03,93442.99,43872.74705,2025-04-22 23:59:59.999,3967785064.12817020,3268475,22777.77047,2062639579.20308550,0,2025-04-22,5926.77,6.772184,2.491293,86381.207143,83555.246667,7061.782857
26,2025-04-23,93442.99,94696.05,91935.41,93691.08,27404.16808,2025-04-23 23:59:59.999,2560279933.94795160,3016536,13128.05774,1226992558.57654620,0,2025-04-23,248.09,0.265499,2.507577,87761.307143,84087.379524,5929.772857
27,2025-04-24,93691.07,94005.00,91660.01,93980.47,19497.06071,2025-04-24 23:59:59.999,1811120137.31467870,2797383,8565.18128,795787487.36089810,0,2025-04-24,289.40,0.308877,2.550737,89051.672857,84600.111905,4928.797143


In [54]:
# RSI (Relative Strength Index) - Индекс относительной силы 
# RSI > 70 → цена может упасть, RSI < 30 → цена может вырасти.

from ta.momentum import RSIIndicator  
rsi = RSIIndicator(klines_df['close'], window=14)  
klines_df['RSI'] = rsi.rsi()  

# MACD (Moving Average Convergence Divergence) – тренд и моменты разворота.
from ta.trend import MACD  
macd = MACD(klines_df['close'])  
klines_df['MACD'] = macd.macd()  
klines_df['MACD_signal'] = macd.macd_signal()  

# Bollinger Bands – волатильность и границы диапазона.
from ta.volatility import BollingerBands  
bb = BollingerBands(klines_df['close'])  
klines_df['BB_upper'] = bb.bollinger_hband()  
klines_df['BB_lower'] = bb.bollinger_lband()  
klines_df.tail(7)

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,...,pct_change,volatility_7d,MA_7,MA_21,price_ma7_diff,RSI,MACD,MACD_signal,BB_upper,BB_lower
21,2025-04-18,84947.92,85132.08,84303.96,84474.69,6529.96315,2025-04-18 23:59:59.999,552648483.64748710,840009,2911.78856,...,-0.557071,1.402898,84389.350000,82675.790000,85.340000,53.353817,NaN,NaN,87416.000270,77938.304730
22,2025-04-19,84474.70,85677.99,84364.45,85077.01,9666.58153,2025-04-19 23:59:59.999,823222752.28424930,875226,5416.30087,...,0.713018,1.127200,84360.794286,82791.431429,716.215714,54.903858,NaN,NaN,87661.230107,77961.776893
23,2025-04-20,85077.00,85320.76,83949.52,85179.24,8091.67725,2025-04-20 23:59:59.999,685253644.89183620,944130,3843.48314,...,0.120162,0.823125,84563.542857,82924.252857,615.697143,55.176111,NaN,NaN,87898.596051,77987.333949
24,2025-04-21,85179.24,88465.99,85144.76,87516.23,31773.37262,2025-04-21 23:59:59.999,2773299591.46706560,3041395,17216.39877,...,2.743615,1.246152,84981.350000,83160.739524,2534.880000,60.976097,NaN,NaN,88324.323753,77797.395247
25,2025-04-22,87516.22,93888.00,87076.03,93442.99,43872.74705,2025-04-22 23:59:59.999,3967785064.12817020,3268475,22777.77047,...,6.772184,2.491293,86381.207143,83555.246667,7061.782857,71.165955,1334.899994,NaN,90536.014296,76678.374704
26,2025-04-23,93442.99,94696.05,91935.41,93691.08,27404.16808,2025-04-23 23:59:59.999,2560279933.94795160,3016536,13128.05774,...,0.265499,2.507577,87761.307143,84087.379524,5929.772857,71.501412,1857.962345,NaN,92329.663796,75932.524204
27,2025-04-24,93691.07,94005.00,91660.01,93980.47,19497.06071,2025-04-24 23:59:59.999,1811120137.31467870,2797383,8565.18128,...,0.308877,2.550737,89051.672857,84600.111905,4928.797143,71.911912,2269.680911,NaN,93887.042720,75384.205280


In [55]:
# Средний объем за N дней
klines_df['volume_ma7'] = klines_df['volume'].rolling(7).mean()  

# Объемный профиль (отношение buy/sell объема)
klines_df['buy_volume_ratio'] = klines_df['taker_buy_base_asset_volume'] / klines_df['volume']  

# Аномалии объема (Z-score)
klines_df['volume_zscore'] = (klines_df['volume'] - klines_df['volume'].mean()) / klines_df['volume'].std()  
klines_df.tail(7)

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,...,MA_21,price_ma7_diff,RSI,MACD,MACD_signal,BB_upper,BB_lower,volume_ma7,buy_volume_ratio,volume_zscore
21,2025-04-18,84947.92,85132.08,84303.96,84474.69,6529.96315,2025-04-18 23:59:59.999,552648483.64748710,840009,2911.78856,...,82675.790000,85.340000,53.353817,NaN,NaN,87416.000270,77938.304730,19120.990143,0.445912,-1.189153
22,2025-04-19,84474.70,85677.99,84364.45,85077.01,9666.58153,2025-04-19 23:59:59.999,823222752.28424930,875226,5416.30087,...,82791.431429,716.215714,54.903858,NaN,NaN,87661.230107,77961.776893,17863.252594,0.560312,-1.007591
23,2025-04-20,85077.00,85320.76,83949.52,85179.24,8091.67725,2025-04-20 23:59:59.999,685253644.89183620,944130,3843.48314,...,82924.252857,615.697143,55.176111,NaN,NaN,87898.596051,77987.333949,15493.486229,0.474992,-1.098754
24,2025-04-21,85179.24,88465.99,85144.76,87516.23,31773.37262,2025-04-21 23:59:59.999,2773299591.46706560,3041395,17216.39877,...,83160.739524,2534.880000,60.976097,NaN,NaN,88324.323753,77797.395247,15938.383249,0.541850,0.272052
25,2025-04-22,87516.22,93888.00,87076.03,93442.99,43872.74705,2025-04-22 23:59:59.999,3967785064.12817020,3268475,22777.77047,...,83555.246667,7061.782857,71.165955,1334.899994,NaN,90536.014296,76678.374704,19218.633501,0.519178,0.972420
26,2025-04-23,93442.99,94696.05,91935.41,93691.08,27404.16808,2025-04-23 23:59:59.999,2560279933.94795160,3016536,13128.05774,...,84087.379524,5929.772857,71.501412,1857.962345,NaN,92329.663796,75932.524204,20152.479629,0.479053,0.019143
27,2025-04-24,93691.07,94005.00,91660.01,93980.47,19497.06071,2025-04-24 23:59:59.999,1811120137.31467870,2797383,8565.18128,...,84600.111905,4928.797143,71.911912,2269.680911,NaN,93887.042720,75384.205280,20976.510056,0.439306,-0.438557


In [56]:
# День недели
klines_df['day_of_week'] = klines_df['date'].dt.dayofweek  
klines_df.tail(7)

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,...,price_ma7_diff,RSI,MACD,MACD_signal,BB_upper,BB_lower,volume_ma7,buy_volume_ratio,volume_zscore,day_of_week
21,2025-04-18,84947.92,85132.08,84303.96,84474.69,6529.96315,2025-04-18 23:59:59.999,552648483.64748710,840009,2911.78856,...,85.340000,53.353817,NaN,NaN,87416.000270,77938.304730,19120.990143,0.445912,-1.189153,4
22,2025-04-19,84474.70,85677.99,84364.45,85077.01,9666.58153,2025-04-19 23:59:59.999,823222752.28424930,875226,5416.30087,...,716.215714,54.903858,NaN,NaN,87661.230107,77961.776893,17863.252594,0.560312,-1.007591,5
23,2025-04-20,85077.00,85320.76,83949.52,85179.24,8091.67725,2025-04-20 23:59:59.999,685253644.89183620,944130,3843.48314,...,615.697143,55.176111,NaN,NaN,87898.596051,77987.333949,15493.486229,0.474992,-1.098754,6
24,2025-04-21,85179.24,88465.99,85144.76,87516.23,31773.37262,2025-04-21 23:59:59.999,2773299591.46706560,3041395,17216.39877,...,2534.880000,60.976097,NaN,NaN,88324.323753,77797.395247,15938.383249,0.541850,0.272052,0
25,2025-04-22,87516.22,93888.00,87076.03,93442.99,43872.74705,2025-04-22 23:59:59.999,3967785064.12817020,3268475,22777.77047,...,7061.782857,71.165955,1334.899994,NaN,90536.014296,76678.374704,19218.633501,0.519178,0.972420,1
26,2025-04-23,93442.99,94696.05,91935.41,93691.08,27404.16808,2025-04-23 23:59:59.999,2560279933.94795160,3016536,13128.05774,...,5929.772857,71.501412,1857.962345,NaN,92329.663796,75932.524204,20152.479629,0.479053,0.019143,2
27,2025-04-24,93691.07,94005.00,91660.01,93980.47,19497.06071,2025-04-24 23:59:59.999,1811120137.31467870,2797383,8565.18128,...,4928.797143,71.911912,2269.680911,NaN,93887.042720,75384.205280,20976.510056,0.439306,-0.438557,3


In [57]:
# Бычье/медвежье поглощение
klines_df['bullish_engulfing'] = (
    (klines_df['close'] > klines_df['open']) & 
    (klines_df['close'].shift(1) < klines_df['open'].shift(1)) & 
    (klines_df['close'] > klines_df['open'].shift(1)) & 
    (klines_df['open'] < klines_df['close'].shift(1))
).astype(int)  

# Доджи (нерешительность рынка)
klines_df['doji'] = (abs(klines_df['open'] - klines_df['close']) / (klines_df['high'] - klines_df['low']) < 0.1).astype(int)  
klines_df.tail(7)

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,...,MACD,MACD_signal,BB_upper,BB_lower,volume_ma7,buy_volume_ratio,volume_zscore,day_of_week,bullish_engulfing,doji
21,2025-04-18,84947.92,85132.08,84303.96,84474.69,6529.96315,2025-04-18 23:59:59.999,552648483.64748710,840009,2911.78856,...,NaN,NaN,87416.000270,77938.304730,19120.990143,0.445912,-1.189153,4,0,0
22,2025-04-19,84474.70,85677.99,84364.45,85077.01,9666.58153,2025-04-19 23:59:59.999,823222752.28424930,875226,5416.30087,...,NaN,NaN,87661.230107,77961.776893,17863.252594,0.560312,-1.007591,5,0,0
23,2025-04-20,85077.00,85320.76,83949.52,85179.24,8091.67725,2025-04-20 23:59:59.999,685253644.89183620,944130,3843.48314,...,NaN,NaN,87898.596051,77987.333949,15493.486229,0.474992,-1.098754,6,0,1
24,2025-04-21,85179.24,88465.99,85144.76,87516.23,31773.37262,2025-04-21 23:59:59.999,2773299591.46706560,3041395,17216.39877,...,NaN,NaN,88324.323753,77797.395247,15938.383249,0.541850,0.272052,0,0,0
25,2025-04-22,87516.22,93888.00,87076.03,93442.99,43872.74705,2025-04-22 23:59:59.999,3967785064.12817020,3268475,22777.77047,...,1334.899994,NaN,90536.014296,76678.374704,19218.633501,0.519178,0.972420,1,0,0
26,2025-04-23,93442.99,94696.05,91935.41,93691.08,27404.16808,2025-04-23 23:59:59.999,2560279933.94795160,3016536,13128.05774,...,1857.962345,NaN,92329.663796,75932.524204,20152.479629,0.479053,0.019143,2,0,1
27,2025-04-24,93691.07,94005.00,91660.01,93980.47,19497.06071,2025-04-24 23:59:59.999,1811120137.31467870,2797383,8565.18128,...,2269.680911,NaN,93887.042720,75384.205280,20976.510056,0.439306,-0.438557,3,0,0


In [58]:
# Лаговые значения (для предсказания)
klines_df['close_lag1'] = klines_df['close'].shift(1)  
klines_df['close_lag3'] = klines_df['close'].shift(3)  

# Целевая переменная (например, цена через N дней)
klines_df['target_close_3d'] = klines_df['close'].shift(-3)  
klines_df.tail(7)

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,...,BB_lower,volume_ma7,buy_volume_ratio,volume_zscore,day_of_week,bullish_engulfing,doji,close_lag1,close_lag3,target_close_3d
21,2025-04-18,84947.92,85132.08,84303.96,84474.69,6529.96315,2025-04-18 23:59:59.999,552648483.64748710,840009,2911.78856,...,77938.304730,19120.990143,0.445912,-1.189153,4,0,0,84947.91,83643.99,87516.23
22,2025-04-19,84474.70,85677.99,84364.45,85077.01,9666.58153,2025-04-19 23:59:59.999,823222752.28424930,875226,5416.30087,...,77961.776893,17863.252594,0.560312,-1.007591,5,0,0,84474.69,84030.38,93442.99
23,2025-04-20,85077.00,85320.76,83949.52,85179.24,8091.67725,2025-04-20 23:59:59.999,685253644.89183620,944130,3843.48314,...,77987.333949,15493.486229,0.474992,-1.098754,6,0,1,85077.01,84947.91,93691.08
24,2025-04-21,85179.24,88465.99,85144.76,87516.23,31773.37262,2025-04-21 23:59:59.999,2773299591.46706560,3041395,17216.39877,...,77797.395247,15938.383249,0.541850,0.272052,0,0,0,85179.24,84474.69,93980.47
25,2025-04-22,87516.22,93888.00,87076.03,93442.99,43872.74705,2025-04-22 23:59:59.999,3967785064.12817020,3268475,22777.77047,...,76678.374704,19218.633501,0.519178,0.972420,1,0,0,87516.23,85077.01,NaN
26,2025-04-23,93442.99,94696.05,91935.41,93691.08,27404.16808,2025-04-23 23:59:59.999,2560279933.94795160,3016536,13128.05774,...,75932.524204,20152.479629,0.479053,0.019143,2,0,1,93442.99,85179.24,NaN
27,2025-04-24,93691.07,94005.00,91660.01,93980.47,19497.06071,2025-04-24 23:59:59.999,1811120137.31467870,2797383,8565.18128,...,75384.205280,20976.510056,0.439306,-0.438557,3,0,0,93691.08,87516.23,NaN


In [59]:
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands
def get_klines(symbol: str, start_date: str, end_date: str, interval: str = "1d") -> pd.DataFrame:
    sdt = int(datetime.strptime(start_date, DT_FORMAT).timestamp() * 1000)
    edt = int(datetime.strptime(end_date, DT_FORMAT).timestamp() * 1000)

    params = {
        "symbol": symbol,
        "interval": interval,
        "startTime": sdt,
        "endTime": edt
    }

    klines = requests.get('https://api.binance.com/api/v3/klines', params=params)
    klines_df = pd.DataFrame(klines.json())

    klines_df.columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume',
                         'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
    klines_df['open_time'] = pd.to_datetime(klines_df['open_time'], unit='ms')
    klines_df['close_time'] = pd.to_datetime(klines_df['close_time'], unit='ms')
    for col in ['open', 'high', 'low', 'close', 'volume']:
        klines_df[col] = klines_df[col].apply(lambda x: float(x))

    klines_df['date'] = klines_df['open_time']
    # klines_df = klines_df[['date', 'close_time', 'volume', 'open', 'close']]
    klines_df['diff'] = klines_df['close'] - klines_df['open']
    klines_df['taker_buy_base_asset_volume'] = klines_df['taker_buy_base_asset_volume'].astype('float')
    
    # Процентное изменение цены
    klines_df['pct_change'] = klines_df['close'].pct_change() * 100  
    
    # Среднеисторическая волатильность (скользящее стандартное отклонение)
    klines_df['volatility_7d'] = klines_df['pct_change'].rolling(7).std()  
    
    # Скользящие средние (MA)
    klines_df['MA_7'] = klines_df['close'].rolling(7).mean()  
    klines_df['MA_21'] = klines_df['close'].rolling(21).mean()  
    
    # Разница между ценой и MA (отклонение от тренда)
    klines_df['price_ma7_diff'] = klines_df['close'] - klines_df['MA_7']

    # RSI (Relative Strength Index) - Индекс относительной силы, RSI > 70 → цена может упасть, RSI < 30 → цена может вырасти.
    rsi = RSIIndicator(klines_df['close'], window=14)
    klines_df['RSI'] = rsi.rsi()
    
    # MACD (Moving Average Convergence Divergence) – тренд и моменты разворота.
    macd = MACD(klines_df['close'])
    klines_df['MACD'] = macd.macd()
    klines_df['MACD_signal'] = macd.macd_signal()
    
    # Bollinger Bands – волатильность и границы диапазона.
    bb = BollingerBands(klines_df['close'])
    klines_df['BB_upper'] = bb.bollinger_hband()
    klines_df['BB_lower'] = bb.bollinger_lband()
    
    # Средний объем за N дней
    klines_df['volume_ma7'] = klines_df['volume'].rolling(7).mean()  
    
    # Объемный профиль (отношение buy/sell объема)
    klines_df['buy_volume_ratio'] = klines_df['taker_buy_base_asset_volume'] / klines_df['volume']  
    
    # Аномалии объема (Z-score)
    klines_df['volume_zscore'] = (klines_df['volume'] - klines_df['volume'].mean()) / klines_df['volume'].std()  
    
    klines_df['day_of_week'] = klines_df['date'].dt.dayofweek  
    
    # Бычье/медвежье поглощение
    klines_df['bullish_engulfing'] = (
        (klines_df['close'] > klines_df['open']) & 
        (klines_df['close'].shift(1) < klines_df['open'].shift(1)) & 
        (klines_df['close'] > klines_df['open'].shift(1)) & 
        (klines_df['open'] < klines_df['close'].shift(1))
    ).astype(int)  
    
    # Доджи (нерешительность рынка)
    klines_df['doji'] = (abs(klines_df['open'] - klines_df['close']) / (klines_df['high'] - klines_df['low']) < 0.1).astype(int) 
    
    # Лаговые значения (для предсказания)
    klines_df['close_lag1'] = klines_df['close'].shift(1)  
    klines_df['close_lag3'] = klines_df['close'].shift(3)  
    
    # Целевая переменная (например, цена через N дней)
    klines_df['target_close_3d'] = klines_df['close'].shift(-3)  
    return klines_df


end_date = '2025-04-25'
start_date = add_days(end_date, -28)
klines_df = get_klines('BTCUSDT', start_date, end_date)
klines_df.tail(7)

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,...,BB_lower,volume_ma7,buy_volume_ratio,volume_zscore,day_of_week,bullish_engulfing,doji,close_lag1,close_lag3,target_close_3d
21,2025-04-18,84947.92,85132.08,84303.96,84474.69,6529.96315,2025-04-18 23:59:59.999,552648483.64748710,840009,2911.78856,...,77938.304730,19120.990143,0.445912,-1.189153,4,0,0,84947.91,83643.99,87516.23
22,2025-04-19,84474.70,85677.99,84364.45,85077.01,9666.58153,2025-04-19 23:59:59.999,823222752.28424930,875226,5416.30087,...,77961.776893,17863.252594,0.560312,-1.007591,5,0,0,84474.69,84030.38,93442.99
23,2025-04-20,85077.00,85320.76,83949.52,85179.24,8091.67725,2025-04-20 23:59:59.999,685253644.89183620,944130,3843.48314,...,77987.333949,15493.486229,0.474992,-1.098754,6,0,1,85077.01,84947.91,93691.08
24,2025-04-21,85179.24,88465.99,85144.76,87516.23,31773.37262,2025-04-21 23:59:59.999,2773299591.46706560,3041395,17216.39877,...,77797.395247,15938.383249,0.541850,0.272052,0,0,0,85179.24,84474.69,93980.47
25,2025-04-22,87516.22,93888.00,87076.03,93442.99,43872.74705,2025-04-22 23:59:59.999,3967785064.12817020,3268475,22777.77047,...,76678.374704,19218.633501,0.519178,0.972420,1,0,0,87516.23,85077.01,NaN
26,2025-04-23,93442.99,94696.05,91935.41,93691.08,27404.16808,2025-04-23 23:59:59.999,2560279933.94795160,3016536,13128.05774,...,75932.524204,20152.479629,0.479053,0.019143,2,0,1,93442.99,85179.24,NaN
27,2025-04-24,93691.07,94005.00,91660.01,93980.47,19497.06071,2025-04-24 23:59:59.999,1811120137.31467870,2797383,8565.18128,...,75384.205280,20976.510056,0.439306,-0.438557,3,0,0,93691.08,87516.23,NaN
